In [2]:
!pip install --quiet azure-ai-projects==1.0.0b9 azure-identity azure-ai-evaluation dotenv

In [3]:
import dotenv
dotenv.load_dotenv()

True

In [4]:

import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import FunctionTool, ToolSet

# Import your custom functions to be used as Tools for the Agent
from user_functions import user_functions

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

AGENT_NAME = "Seattle Tourist Assistant"

# Add Tools to be used by Agent
functions = FunctionTool(user_functions)

toolset = ToolSet()
toolset.add(functions)

# To enable tool calls executed automatically
project_client.agents.enable_auto_function_calls(toolset=toolset)

### Create an AI agent (Azure AI Agent Service)

In [5]:
agent = project_client.agents.create_agent(
    model=os.environ["AGENT_MODEL_DEPLOYMENT_NAME"],
    name=AGENT_NAME,
    instructions="You are a helpful assistant",
    toolset=toolset,
)

print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_EMQwuo5sRsnhWvt6Rn04QzbL


In [6]:
# Create a thread for the agent to interact with
thread = project_client.agents.create_thread()
print(f"Created thread, ID: {thread.id}")

# Create message to thread

MESSAGE = "Can you email me weather info for Seattle ?"

message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=MESSAGE,
)
print(f"Created message, ID: {message.id}")

Created thread, ID: thread_HxbQyc3p6zTajcHnKpY5FmHY
Created message, ID: msg_yqoPRUeqi9bhStKNhJDewlSB


In [7]:
### Execute the Agent run
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)

print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

print(f"Run ID: {run.id}")

Sending email to your_email@example.com...
Subject: Weather Info for Seattle
Body:
The current weather in Seattle is Rainy, with a temperature of 14°C.
Run finished with status: RunStatus.COMPLETED
Run ID: run_aT8FUZr6Etdl9vB63lCIbIjr


In [8]:
for message in project_client.agents.list_messages(thread.id, order="asc").data:
    print(f"Role: {message.role}")
    print(f"Content: {message.content[0].text.value}")
    print("-" * 40)

Role: MessageRole.USER
Content: Can you email me weather info for Seattle ?
----------------------------------------
Role: MessageRole.AGENT
Content: I have sent the weather information for Seattle to your email. If you need any more assistance, feel free to ask!
----------------------------------------


# Evaluate

### Get data from agent

In [9]:
from azure.ai.evaluation import AIAgentConverter
import json


# Initialize the converter that will be backed by the project.
converter = AIAgentConverter(project_client)

thread_id = thread.id
run_id = run.id
file_name = "evaluation_data.jsonl"

# Get a single agent run data
evaluation_data_single_run = converter.convert(thread_id=thread_id, run_id=run_id)